In [18]:
import pandas as pd
import numpy as np
from scipy import stats

# Cargar datos (si no los tienes en memoria)
df = pd.read_csv("../01_Data/raw/onboarding_ab_test_data.csv")

print("--- 1. TEST DE CONVERSIÓN (CHI-SQUARE) ---")
# Tabla de contingencia: Filas=Grupo, Columnas=Convirtió/NoConvirtió
# Queremos ver si 'ab_group' y 'is_converted' son independientes
contingency_table = pd.crosstab(df['ab_group'], df['is_converted'])
print(contingency_table)

# Ejecutamos el test
chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)

print(f"\nValor p (p-value): {p_value:.10f}")
if p_value < 0.05:
    print("✅ RESULTADO: La diferencia es ESTADÍSTICAMENTE SIGNIFICATIVA. Rechazamos la Hipótesis Nula.")
    print("Conclusión: La Variante B realmente funciona mejor, no es ruido.")
else:
    print("❌ RESULTADO: No hay evidencia suficiente. La diferencia podría ser suerte.")

conversion_variante=df[df['ab_group'] == 'Variant']['is_converted'].mean()
conversion_control=df[df['ab_group']=='Control']['is_converted'].mean()

# Calculamos el lift para ver la mejora.
lift = ((conversion_variante - conversion_control) / conversion_control)*100

print(f"La variante incrementó la conversión en {lift:.1f}%")

print("\n--- 2. TEST DE TIEMPOS ANDROID vs iOS (T-TEST) ---")
# Filtramos los tiempos de escaneo (quitando nulos)
android_times = df[df['device_os'] == 'Android']['dni_scan_duration'].dropna()
ios_times = df[df['device_os'] == 'iOS']['dni_scan_duration'].dropna()

# Ejecutamos T-Test de dos muestras independientes
t_stat, p_val_time = stats.ttest_ind(android_times, ios_times, equal_var=False)

print(f"Tiempo Promedio Android: {android_times.mean():.2f}s")
print(f"Tiempo Promedio iOS: {ios_times.mean():.2f}s")
print(f"Valor p (p-value): {p_val_time:.10f}")

if p_val_time < 0.05:
    print("✅ RESULTADO: La lentitud de Android NO es casualidad. Es un problema sistémico probado.")

# Hallamos el Dropoff
# El drop-off es el porcentaje de usuarios que abandonan en un paso del funnel antes de pasar al siguiente.
# Filtrando los usuarios de cada SO
android_users = df[df['device_os']=='Android']
ios_users = df[df['device_os']=='iOS']

## Contamos cuántos llegaron al paso 'DNI_Scan' (o pasaron de él)
llegaron_dni_Android = android_users[android_users['step_reached'].isin(['DNI_Scan', 'Contract', 'Success'])].shape[0]
llegaron_dni_IOS = ios_users[ios_users['step_reached'].isin(['DNI_Scan','Contract','Success'])].shape[0]

# Contamos cuántos SUPERARON el paso (llegaron a Contract o Success)
pasaron_dni_Android = android_users[android_users['step_reached'].isin(['Contract', 'Success'])].shape[0]
pasaron_dni_IOS = ios_users[ios_users['step_reached'].isin(['Contract','Success'])].shape[0]

#Calculamos el DROPOFF
drop_off_rate_Android = (1-(pasaron_dni_Android/llegaron_dni_Android))*100
drop_off_rate_IOS = (1-(pasaron_dni_IOS/llegaron_dni_IOS))*100

print(f"Tasa de abandono en DNI (Android): {drop_off_rate_Android:.2f}%")
print(f"Tasa de abandono en DNI (IOS): {drop_off_rate_IOS:.2f}%")





--- 1. TEST DE CONVERSIÓN (CHI-SQUARE) ---
is_converted     0     1
ab_group                
Control       2558  2420
Variant       2298  2724

Valor p (p-value): 0.0000000203
✅ RESULTADO: La diferencia es ESTADÍSTICAMENTE SIGNIFICATIVA. Rechazamos la Hipótesis Nula.
Conclusión: La Variante B realmente funciona mejor, no es ruido.
La variante incrementó la conversión en 11.6%

--- 2. TEST DE TIEMPOS ANDROID vs iOS (T-TEST) ---
Tiempo Promedio Android: 17.95s
Tiempo Promedio iOS: 11.84s
Valor p (p-value): 0.0000000000
✅ RESULTADO: La lentitud de Android NO es casualidad. Es un problema sistémico probado.
Tasa de abandono en DNI (Android): 10.13%
Tasa de abandono en DNI (IOS): 9.40%
